In [3]:
import basedosdados as bd
import geopandas as gpd
from shapely import wkt

In [30]:
# dados de Nascidos Vivos

query = """
SELECT ano, id_municipio_nascimento, data_nascimento, sexo, peso, raca_cor, apgar1, apgar5, gestacao_agr, tipo_gravidez, tipo_parto,
        quantidade_filhos_vivos, quantidade_filhos_mortos, idade_mae, idade_pai, escolaridade_mae, estado_civil_mae, quantidade_parto_normal,  
        quantidade_parto_cesareo, ocupacao_mae, formacao_profissional_responsavel, tipo_apresentacao, inducao_parto, gestacoes_ant,inicio_pre_natal,
        pre_natal, id_anomalia, codigo_anomalia, 

FROM basedosdados.br_ms_sinasc.microdados
WHERE (ano BETWEEN 2010 AND 2020) AND (sigla_uf = 'RJ')
"""

df = bd.read_sql(query, billing_project_id="nascidosvivos")

Downloading:   0%|          | 0/2423086 [00:00<?, ?rows/s]

KeyboardInterrupt: 

In [1]:
df.to_csv('data/df.csv', index=True)

NameError: name 'df' is not defined

In [15]:
# dados das cidades do estado do Rio de Janeiro

query = """
SELECT DISTINCT M.id_municipio, SC.nome_municipio, M.geometria
FROM basedosdados.br_geobr_mapas.municipio M
INNER JOIN basedosdados.br_geobr_mapas.setor_censitario_2010 SC
ON M.id_municipio = SC.id_municipio
WHERE SC.sigla_uf = 'RJ'
"""

cidades = bd.read_sql(query, billing_project_id="nascidosvivos")

Downloading: 100%|██████████| 92/92 [00:01<00:00, 62.29rows/s]


In [16]:
cidades['geometria'] = cidades['geometria'].apply(wkt.loads)
cidades = gpd.GeoDataFrame(cidades, geometry='geometria')

cidades['id_municipio'] = cidades['id_municipio'].astype(int)
cidades['id_municipio'] = cidades['id_municipio'].astype(str)

cidades.to_file('data/cidades.geojson', driver='GeoJSON')
cidades.to_csv('data/cidades.csv', index=True)

cidades.to_file('../Nascidos-Vivos-Viz/cidades.geojson', driver='GeoJSON')